In [1]:
import math
import numpy as np
import torch
from datasets import load_dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaTokenizerFast, RobertaConfig, RobertaModelWithHeads
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaForMaskedLM
from transformers import AdapterType
from sklearn.metrics import f1_score

In [2]:
import datetime

In [3]:
today = datetime.date.today().strftime('%Y%m%d')

In [4]:
def check_gpu():
    '''
    check gpu status
    '''
    try:
        print('GPU available:', torch.cuda.is_available())
        print(torch.cuda.device_count(), 'GPUs detected')
        print('Current GPU id:', torch.cuda.current_device())
        print('Current GPU Name:', torch.cuda.get_device_name(torch.cuda.current_device()))
    except:
        print('GPU not available')

In [5]:
def encode_batch(batch):
    '''
    Encodes a batch of input data using the model tokenizer
    using 512
    '''
    return tokenizer(batch["text"], max_length=120, truncation=True, padding="max_length")
#     return tokenizer(batch["text"], padding=True, truncation=True)

In [6]:
check_gpu()

GPU available: True
1 GPUs detected
Current GPU id: 0
Current GPU Name: GeForce RTX 2070 with Max-Q Design


In [7]:
# dictionary for dataset, name: (classes, type of f1 score)
dataset_dict = {'chemprot': (13, 'micro'), 'rct': (5, 'micro'),
                'CI': (6, 'macro'), 'sciie': (7, 'm2cro'),
                'HN': (2, 'macro'), 'ag': (4, 'macro'),
                'amazon': (2, 'macro'), 'imdb': (2, 'macro')}

In [8]:
ds_name = 'amazon'
n_labels = dataset_dict[ds_name][0]
f1_type = dataset_dict[ds_name][1]

In [9]:
dataset = load_dataset(f'data_loaders/{ds_name}_data_loader.py')

Reusing dataset task_dataset (C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae)


In [10]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [11]:
#dataset_encoded = dataset.map(encode_batch, batched=True, batch_size=512, remove_columns=["text"])
dataset_encoded = dataset.map(encode_batch, batched=True)
# tokenized_datasets = dataset.map(tokenizer, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-294b1a221a18a2c0.arrow
Loading cached processed dataset at C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-bea7b60a230a39b6.arrow
Loading cached processed dataset at C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-c40cb088f0753106.arrow


In [12]:
dataset_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 115251
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 25000
    })
})

## Case 4
adapter language model

In [14]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.add_adapter('mlm', AdapterType.text_lang)

In [16]:
model.set_active_adapters(['mlm'])

In [17]:
model.train_adapter(["mlm"])

In [18]:
# get warm up steps for given warmup ratio

warmup_ratio = 0.006
max_train_batch_size_mlm = 32
WARMUP_STEP = max(1,int(dataset_encoded['train'].num_rows / max_train_batch_size_mlm * warmup_ratio))
print(WARMUP_STEP)
GRADIENT_ACC_STEP = 256 / max_train_batch_size_mlm
print(GRADIENT_ACC_STEP)

21
8.0


In [19]:
training_args_mlm = TrainingArguments(
    output_dir=f'model/{ds_name}/mlm-adapter/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=0.00025,
    remove_unused_columns=False,
    num_train_epochs=100,
    fp16=True,
    per_device_train_batch_size=max_train_batch_size_mlm,
    per_device_eval_batch_size=32,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=WARMUP_STEP,
    weight_decay=0.01,
    gradient_accumulation_steps=GRADIENT_ACC_STEP
)

In [20]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [21]:
trainer_mlm = Trainer(
    model=model,
    args=training_args_mlm,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    data_collator=data_collator,
)

In [22]:
trainer_mlm.train()

Epoch,Training Loss,Validation Loss
0,No log,1.589875
1,1.790488,1.562097
2,1.672804,1.556694
3,1.635920,1.526788
4,1.615113,1.495778
5,1.601512,1.496184
6,1.591165,1.481291
7,1.577594,1.494827
8,1.569970,1.469940
9,1.563271,1.497905


TrainOutput(global_step=45000, training_loss=1.487103125)

In [23]:
trainer_mlm.evaluate()

{'eval_loss': 1.4022356271743774, 'epoch': 99.99944475291505}

In [24]:
# eval_results = trainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [25]:
model.save_pretrained(f'model/amazon/{today}/model')

In [26]:
model.save_adapter(f'model/amazon/{today}/adapter', 'mlm')